In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.io as pio
from upsetplot import UpSet, from_contents, from_memberships
import plotly.express as px
import pyupset as pyu

In [ ]:
CM1_path = './CM1/final_results/abyss/'
CM3_path = './CM3/final_results/abyss/'

In [70]:
CM1_files = os.listdir(CM1_path)
CM1_files = [file for file in CM1_files if file.endswith('pvals.tsv')]
CM1_files = [file for file in CM1_files if file.startswith('A_contigs_')]
CM1_files = [file for file in CM1_files if file!='A_contigs_pvals.tsv']

In [71]:
CM3_files = os.listdir(CM3_path)
CM3_files = [file for file in CM3_files if file.endswith('pvals.tsv')]
CM3_files = [file for file in CM3_files if file.startswith('A_contigs_')]
CM3_files = [file for file in CM3_files if file!='A_contigs_pvals.tsv']

In [75]:
def draw_upset_plot(df, annotation_type):
    intersection_dict = {}
    for cell in df['cell'].unique():
        intersection_dict[cell] = set(df[df['cell']==cell]['gene_name'])


    data = from_contents(intersection_dict)

    upset = UpSet(data, show_counts=True, orientation='horizontal', sort_categories_by=None)
    upset.plot()

In [ ]:
CM1_merged_df = pd.DataFrame()
CM3_merged_df = pd.DataFrame()
for file in CM1_files:
    ann_type = file.split('_')[2:]
    ann_type = '_'.join(ann_type)
    ann_type = ann_type.split('.')[0]
    ann_type = ann_type.split('_')[0]
    if ann_type not in ['retainedintrons', 'junctions', 'lncRNAs']:
        continue
    CM1 = pd.read_csv(CM1_path+file, sep='\t', index_col=0)
    CM3 = pd.read_csv(CM3_path+file, sep='\t', index_col=0)
    CM1.drop_duplicates(subset=['contig', 'chr', 'start', 'end', 'gene_name'])
    CM3.drop_duplicates(subset=['contig', 'chr', 'start', 'end', 'gene_name'])
    CM1['contig_length'] = CM1['contig'].apply(lambda x: len(x))
    CM3['contig_length'] = CM3['contig'].apply(lambda x: len(x))
    CM1['cell'] = 'CM1'
    CM3['cell'] = 'CM3'
    CM1['annotation_type'] = ann_type
    CM3['annotation_type'] = ann_type
    merged_df = pd.concat([CM1, CM3], axis=0)
    CM1_merged_df = pd.concat([CM1_merged_df, CM1])
    CM3_merged_df = pd.concat([CM3_merged_df, CM3])
    try:
        draw_upset_plot(merged_df, ann_type)
    except:
        pass